In [287]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [288]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development.

In [289]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("./kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [290]:
X = pd.DataFrame(data, columns=["text"])
X

,text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL..."
1,Will do.
2,Nora--Cheryl has emailed dozens of memos about...
3,Dear Sir=2FMadam=2C I know that this proposal ...
4,fyi
...,...
995,So what's the latest? It sounds contradictory ...
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO..."
997,Barb I will call to explain. Are you back in t...
998,Yang on travelNot free tonite.May work tomorrow


In [291]:
y = pd.DataFrame(data['label'])
y

,label
0,1
1,0
2,0
3,1
4,0
...,...
995,0
996,1
997,0
998,0


In [292]:
# Your code
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Preprocessing

In [293]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [294]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [295]:
!pip install beautifulsoup4 lxml

In [296]:
from bs4 import BeautifulSoup, Comment
import re

In [297]:
def clean_html(html):
  #for index, row in X.iterrows():
  soup = BeautifulSoup(html, "lxml")

  for script_or_style in soup(["script", "style"]):
    script_or_style.decompose()

  for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
    comment.extract()

  cleaned_text = soup.get_text(separator=" ", strip=True)

  return cleaned_text

In [298]:
train_data['cleaned_html'] = train_data['text'].apply(clean_html)


<ipython-input-297-a888ae1cc719>:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
<ipython-input-297-a888ae1cc719>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "lxml")
<ipython-input-297-a888ae1cc719>:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html, "lxml")


In [299]:
print(train_data[['text', 'cleaned_html']])

                                                  text  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...   
535  I have not been able to reach oscar this am. W...   
695  ; Huma Abedin B6I'm checking with Pat on the 5...   
557  I can have it announced here on Monday - can't...   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...   
..                                                 ...   
106  7653 2612ADAMA IBRAHIM________________________...   
270             What does that mean for our schedules?   
860  Dear Friend,My Compliment to you,I guess this ...   
435  Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...   
102  Let me know if today or tomorrow works for you...   

                                          cleaned_html  
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...  
535  I have not been able to reach oscar this am. W...  
695  ; Huma Abedin B6I'm checking with Pat on the 5...  
557  I can have it announced here on Monday - can't...  
836  BANK OF AFRIC

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [300]:
# Your code
def clean_text(text):
  # Step 4: Remove all special characters
  cleaned_text = re.sub(r'[^\w\s]', '', text)
  # Step 5: Remove numbers
  cleaned_text = re.sub(r'\d+', '', cleaned_text)
  # Step 6: Remove all single characters
  cleaned_text = re.sub(r'\b\w{1}\b', '', cleaned_text)
  # Step 7: Remove single characters from the start
  cleaned_text = re.sub(r'^\s*\w{1}\s*', '', cleaned_text)
  # Step 8: Substitute multiple spaces with a single space
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
  # Step 9: Remove prefixed 'b'
  cleaned_text = re.sub(r'^b\s*', '', cleaned_text)
  # Step 10: Convert to lowercase
  cleaned_text = cleaned_text.lower()

  return cleaned_text

In [301]:
train_data['cleaned_text'] = train_data['cleaned_html'].apply(clean_html)

<ipython-input-297-a888ae1cc719>:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
<ipython-input-297-a888ae1cc719>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "lxml")
<ipython-input-297-a888ae1cc719>:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html, "lxml")


In [302]:
print(train_data[['text','cleaned_html', 'cleaned_text']])

                                                  text  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...   
535  I have not been able to reach oscar this am. W...   
695  ; Huma Abedin B6I'm checking with Pat on the 5...   
557  I can have it announced here on Monday - can't...   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...   
..                                                 ...   
106  7653 2612ADAMA IBRAHIM________________________...   
270             What does that mean for our schedules?   
860  Dear Friend,My Compliment to you,I guess this ...   
435  Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...   
102  Let me know if today or tomorrow works for you...   

                                          cleaned_html  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...   
535  I have not been able to reach oscar this am. W...   
695  ; Huma Abedin B6I'm checking with Pat on the 5...   
557  I can have it announced here on Monday - can't...   
836  BANK OF 

In [303]:
train_data

,text,cleaned_html,cleaned_text
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...","----------- REGARDS, MR NELSON SMITH.KINDLY RE...","----------- REGARDS, MR NELSON SMITH.KINDLY RE..."
535,I have not been able to reach oscar this am. W...,I have not been able to reach oscar this am. W...,I have not been able to reach oscar this am. W...
695,; Huma Abedin B6I'm checking with Pat on the 5...,; Huma Abedin B6I'm checking with Pat on the 5...,; Huma Abedin B6I'm checking with Pat on the 5...
557,I can have it announced here on Monday - can't...,I can have it announced here on Monday - can't...,I can have it announced here on Monday - can't...
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
...,...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,7653 2612ADAMA IBRAHIM________________________...,7653 2612ADAMA IBRAHIM________________________...
270,What does that mean for our schedules?,What does that mean for our schedules?,What does that mean for our schedules?
860,"Dear Friend,My Compliment to you,I guess this ...","Dear Friend,My Compliment to you,I guess this ...","Dear Friend,My Compliment to you,I guess this ..."
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...


In [304]:
train_data = train_data.drop(columns = ["text", "cleaned_html"])

## Now let's work on removing stopwords
Remove the stopwords.

In [305]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [306]:
# Your code
def remove_stopwords(para):
  stop_words = set(stopwords.words('english'))
  cleaned_text = ' '.join(word for word in para.split() if word not in stop_words)

  return cleaned_text

In [307]:
train_data['cleaned_content'] = train_data['cleaned_text'].apply(remove_stopwords)

In [308]:
train_data = train_data.drop(columns=['cleaned_text'])


In [309]:
train_data

,cleaned_content
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE..."
535,I able reach oscar am. We supposed send pdb 11...
695,; Huma Abedin B6I'm checking Pat 50k work Jack...
557,I announced Monday - can't today
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
...,...
106,7653 2612ADAMA IBRAHIM________________________...
270,What mean schedules?
860,"Dear Friend,My Compliment you,I guess letter m..."
435,Dear PRESIDENT=2FDIRECTOR=2C My name Mr=2E Mic...


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [310]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [311]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [312]:
# Your code
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
  words = nltk.word_tokenize(text)
  lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]
  return ' '.join(lemmatized_words)


In [313]:
train_data['lemmatized_text'] = train_data['cleaned_content'].apply(preprocess_text)

In [314]:
X

,text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL..."
1,Will do.
2,Nora--Cheryl has emailed dozens of memos about...
3,Dear Sir=2FMadam=2C I know that this proposal ...
4,fyi
...,...
995,So what's the latest? It sounds contradictory ...
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO..."
997,Barb I will call to explain. Are you back in t...
998,Yang on travelNot free tonite.May work tomorrow


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [315]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_data_bow = vectorizer.fit_transform(train_data['lemmatized_text'])

top_n = 10

# Convert to DataFrame for better visualization
bow_df = pd.DataFrame(train_data_bow.toarray(), columns=vectorizer.get_feature_names_out())
bow_df['label'] = train_labels

# For ham messages
ham_words = bow_df[bow_df['label'] == 'ham'].drop(columns='label').sum().sort_values(ascending=False).head(top_n)

# For spam messages
spam_words = bow_df[bow_df['label'] == 'spam'].drop(columns='label').sum().sort_values(ascending=False).head(top_n)

# Display top words
print("Top 10 words in Ham messages:")
print(ham_words)
print("\nTop 10 words in Spam messages:")
print(spam_words)


Top 10 words in Ham messages:
00             0
outskirts      0
ov2fh          0
ov             0
oux03ypn       0
ouvrir         0
outweigh       0
outurgently    0
outstanding    0
outsource      0
dtype: int64

Top 10 words in Spam messages:
00             0
outskirts      0
ov2fh          0
ov             0
oux03ypn       0
ouvrir         0
outweigh       0
outurgently    0
outstanding    0
outsource      0
dtype: int64


## Extra features

In [316]:
# # We add to the original dataframe two additional indicators (money symbols and suspicious words).
# money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
# suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

# data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
# data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
# data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

# data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
# data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
# data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

# data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [317]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vectorizer = TfidfVectorizer()

# fit and transform
tfid_matrix = tfid_vectorizer.fit_transform(train_data['lemmatized_text'])

In [318]:
# # Print the shape of the vectorized dataset
# print("Shape of TF-IDF matrix:", tfid_matrix.shape)

# # Optional: to see the matrix contents (dense format for readability, usually sparse)
# print(pd.DataFrame(tfid_matrix.toarray(), columns=tfid_vectorizer.get_feature_names_out()))

In [319]:
train_data = train_data.drop(columns=['cleaned_content'])

In [320]:
train_data

,lemmatized_text
29,"-- -- -- -- -- - REGARDS , MR NELSON SMITH.KIN..."
535,I able reach oscar be . We suppose send pdb 11...
695,; Huma Abedin B6I 'm check Pat 50k work Jack &...
557,I announce Monday - ca n't today
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
...,...
106,7653 2612ADAMA IBRAHIM________________________...
270,What mean schedule ?
860,"Dear Friend , My Compliment you , I guess lett..."
435,Dear PRESIDENT=2FDIRECTOR=2C My name Mr=2E Mic...


## And the Train a Classifier?

In [321]:
# Your code

tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(train_data)
X_test_tfidf = tfidf_vectorizer.transform(test_data)


In [324]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()  # Naive Bayes is a common choice for text data
classifier.fit(X_train_tfidf, train_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Found input variables with inconsistent numbers of samples: [1, 800]

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code